In [11]:
import open3d as o3d
import numpy as np
import cv2 as cv

In [12]:
pcd = o3d.io.read_point_cloud("./pointcloud_raw/scatters/out_10.ply")

In [13]:
pcd

PointCloud with 57229 points.

In [14]:
print(np.asarray(pcd.points).shape)

(57229, 3)


In [15]:
len(pcd.colors)
color = np.asarray(pcd.colors)
color[:,[0,2]] = color[:,[2,0]]
pcd.colors = o3d.utility.Vector3dVector(color)

In [16]:
o3d.visualization.draw_geometries([pcd])

In [17]:
new_color = np.asarray(pcd.colors)
mask = (new_color[:,0] > 0.5) * (new_color[:,1] < 0.4) * (new_color[:,2] < 0.4)
points = np.asarray(pcd.points)
new_pcd = o3d.geometry.PointCloud()
new_pcd.points = o3d.utility.Vector3dVector(points[mask])
new_pcd.colors = o3d.utility.Vector3dVector(new_color[mask])

In [18]:
o3d.visualization.draw_geometries([new_pcd])

In [8]:
def computeTF(pointcloud_register):
    # Remove outlier
    pointcloud_register.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.04)
    # Compute obb
    points = np.asarray(pointcloud_register.points)
    top = points[np.argmax(points[:,1])]
    bottom = points[np.argmin(points[:,1])]
    left = points[np.argmin(points[:,0])]
    right = points[np.argmax(points[:,0])]
    
    R = np.zeros((3,3))
    R[:,0] = (right - left)/np.linalg.norm(right-left,2)
    R[:,1] = (top - bottom)/np.linalg.norm(top - bottom,2)
    R[:,2] = np.cross(R[:,0],R[:,1])
    R = R.T

    # set the point with minimum x then minimum y as origin
    T = -(top + bottom+left+right)/4
    return R, T
# Pw = R(Pc + T)

In [9]:
new_color = np.asarray(pcd.colors)
R, T =  computeTF(new_pcd)
new_pcd.translate(T)
new_pcd.rotate(R, center=(0, 0, 0))


PointCloud with 360 points.

In [10]:
o3d.visualization.draw_geometries([new_pcd])

In [ ]:
np.savez("extrinsic.npz", R=R, T=T)